In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

# Leer csv
df_tiendas = pd.read_csv('/content/drive/MyDrive/orders.csv')

Mounted at /content/drive


<ipython-input-2-46a1a4f85d36>:7: DtypeWarning: Columns (12,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tiendas = pd.read_csv('/content/drive/MyDrive/orders.csv')


In [ ]:
df_tiendas.columns

Index(['Unnamed: 0', 'id', 'supplier_order_id', 'order_id', 'ship_at',
       'shipped_at', 'supplier_id', 'customer_id', 'canceler',
       'supplier_reason', 'supplier_comment', 'customer_reason',
       'customer_comment', 'total', 'discount_amount', 'created_at',
       'confirmed_at', 'canceled_at', 'updated_at', 'credits_paid',
       'money_paid', 'service_fee', 'discount_id', 'delivery_lat',
       'delivery_lon', 'delivery_cost', 'final_amount', 'formatted_address',
       'customer_type_code', 'additional_info', 'promotions_amount',
       'state_id', 'postal_code'],
      dtype='object')

# **FINAL AMOUNT**

In [ ]:
fechas = pd.date_range(start='2021-01-01', end='2024-09-01', freq='M')
print(len(fechas))

44


In [ ]:
variaciones = [
    4, 3.6, 4.8, 4.1, 3.3, 3.2, 3, 2.5, 3.5, 3.5, 2.5, 3.8,  # 2021
    3.9, 4.7, 6.7, 6, 5.1, 5.3, 7.4, 7, 6.2, 6.3, 4.9, 5.1,  # 2022
    6, 6.6, 7.7, 8.4, 7.8, 6, 6.3, 12.4, 12.7, 8.3, 12.8, 25.5,  # 2023
    20.6, 13.2, 11, 8.8, 4.2, 4.6, 4, 4.2  # 2024 (hasta agosto)
]
print(len(variaciones))

44


In [ ]:
# Ajuste por inflación con la variación del IPC
ipc = pd.DataFrame({
    'fecha': fechas,
    'variacion': variaciones
})

In [ ]:
ipc['fecha'].max()

Timestamp('2024-08-31 00:00:00')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar los datos
items = pd.read_csv("/content/drive/MyDrive/items.csv")
orders = pd.read_csv("/content/drive/MyDrive/orders.csv")
customers = pd.read_csv("/content/drive/MyDrive/customers.csv")

# Total de monto de pedido por cada customer
pedidos = orders[['customer_id', 'total', 'created_at']].copy()

ipc['variacion'] /= 100
ipc['indice'] = (1 + ipc['variacion']).cumprod()

indice_final = ipc.loc[ipc['fecha'] == '2024-08-01', 'indice'].values[0]

pedidos['created_at'] = pd.to_datetime(pedidos['created_at'])
pedidos['mes_inicio'] = pedidos['created_at'].dt.to_period('M').dt.to_timestamp()

pedidos = pedidos.merge(ipc, left_on='mes_inicio', right_on='fecha')
pedidos['adjusted_total'] = pedidos['total'] * (indice_final / pedidos['indice'])

# Graficar
pedidos_grouped = pedidos.groupby('customer_id').agg(promedio_por_pedido=('adjusted_total', 'mean')).reset_index()
plt.figure(figsize=(10, 6))
plt.boxplot(pedidos_grouped['promedio_por_pedido'])
plt.title("Monto promedio de pedido por tienda")
plt.ylabel("Promedio por Pedido")

# Eliminando el outlier
pedidos_grouped = pedidos_grouped[pedidos_grouped['promedio_por_pedido'] < 3000000]
plt.figure(figsize=(10, 6))
plt.boxplot(pedidos_grouped['promedio_por_pedido'])
plt.title("Monto promedio de pedido por tienda")
plt.ylabel("Promedio por Pedido")

# Añadir líneas para Q1, Mediana, Q3
q1 = np.percentile(pedidos_grouped['promedio_por_pedido'], 25)
mediana = np.median(pedidos_grouped['promedio_por_pedido'])
q3 = np.percentile(pedidos_grouped['promedio_por_pedido'], 75)
iqr = q3 - q1

plt.axhline(y=q1, color='red', linestyle='--', label=f'Q1: {q1:.2f}')
plt.axhline(y=mediana, color='blue', linestyle='--', label=f'Mediana: {mediana:.2f}')
plt.axhline(y=q3, color='green', linestyle='--', label=f'Q3: {q3:.2f}')
plt.legend()

plt.show()

print(f"Q1: {q1:.2f}")
print(f"Mediana: {mediana:.2f}")
print(f"Q3: {q3:.2f}")
print(f"IQR: {iqr:.2f}")

<ipython-input-22-530ca0ae0079>:7: DtypeWarning: Columns (12,32) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv("/content/drive/MyDrive/orders.csv")


IndexError: index 0 is out of bounds for axis 0 with size 0

# **CANTIDAD DE ÓRDENES**

In [ ]:
# Calcular la cantidad de órdenes totales por customer_id
ordenes_por_customer = df_tiendas.groupby('customer_id').size().reset_index(name='cantidad_ordenes')

# Combinar la nueva información con el DataFrame original
df_tiendas = df_tiendas.merge(ordenes_por_customer, on='customer_id', how='left')

# **CANTIDAD DE SUPPLIERS**

In [ ]:
# Calcular la cantidad de proveedores únicos por customer_id
proveedores_por_customer = df_tiendas.groupby('customer_id')['supplier_id'].nunique().reset_index(name='cantidad_proveedores')

# Combinar la nueva información con el DataFrame original
df_tiendas = df_tiendas.merge(proveedores_por_customer, on='customer_id', how='left')

# **% DE PEDIDOS HECHOS AL SUPPLIER QUE MÁS LE PIDIÓ**

In [ ]:
# 1. Calcular la cantidad de pedidos por supplier_id para cada customer_id
pedidos_por_supplier = df_tiendas.groupby(['customer_id', 'supplier_id']).size().reset_index(name='cantidad_pedidos')

# 2. Encontrar el supplier con la mayor cantidad de pedidos por cada customer_id
max_pedidos_por_customer = pedidos_por_supplier.groupby('customer_id')['cantidad_pedidos'].max().reset_index(name='max_pedidos')

# 3. Calcular el total de pedidos por customer_id
total_pedidos_por_customer = df_tiendas.groupby('customer_id').size().reset_index(name='total_pedidos')

# 4. Unir max_pedidos_por_customer y total_pedidos_por_customer
max_pedidos_por_customer = max_pedidos_por_customer.merge(total_pedidos_por_customer, on='customer_id')

# 5. Calcular el porcentaje que representa el supplier con más pedidos para cada customer_id
max_pedidos_por_customer['porcentaje_max_supplier'] = (max_pedidos_por_customer['max_pedidos'] / max_pedidos_por_customer['total_pedidos']) * 100

# 6. Unir esta información al DataFrame original
df_tiendas = df_tiendas.merge(max_pedidos_por_customer[['customer_id', 'porcentaje_max_supplier']], on='customer_id', how='left')

# Ver las primeras filas del DataFrame actualizado
print(df_tiendas.head())

   Unnamed: 0                                                 id  \
0          17  bytearray(b'\x00\x00w\x17\xae<@\xb4\x87-\x98\x...   
1          24  bytearray(b'\x00\x00\xb2\x8cP\x03D5\x85\xd0\x1...   
2          63  bytearray(b'\x00\x01\xaaN\xed\x19O<\x8d\xbfwT\...   
3         101  bytearray(b'\x00\x02\x97\xde\xd0\x99NP\x9dG\xd...   
4         153  bytearray(b'\x00\x04-\x18w\x0fE\xae\xa2\xbd3\x...   

   supplier_order_id  order_id     ship_at           shipped_at  supplier_id  \
0            2871968   2667640         NaN                  NaN         1403   
1             860931    793088  2022-01-31  2022-01-31 17:07:16            3   
2            1053490    967693  2022-03-09  2022-03-09 11:22:46            3   
3             608609    561574  2021-12-13  2021-12-13 20:26:10          408   
4            1768817   1630869  2022-06-29  2022-06-29 20:35:39          448   

   customer_id canceler supplier_reason  ... final_amount  \
0        26429      NaN             NaN  ...     

# **PROMEDIO MENSUAL DE ÓRDENES**

In [ ]:
# 1. Asegurarse de que la columna de fechas esté en formato datetime
df_tiendas['created_at'] = pd.to_datetime(df_tiendas['created_at'])

# 2. Extraer el año y mes de la columna 'created_at'
df_tiendas['year_month'] = df_tiendas['created_at'].dt.to_period('M')

# 3. Contar la cantidad de órdenes por customer_id y por mes
ordenes_por_mes = df_tiendas.groupby(['customer_id', 'year_month']).size().reset_index(name='cantidad_ordenes_mes')

# 4. Calcular el promedio de órdenes por mes para cada customer_id
promedio_ordenes_por_mes = ordenes_por_mes.groupby('customer_id')['cantidad_ordenes_mes'].mean().reset_index(name='promedio_ordenes_mes')

# 5. Unir esta información al DataFrame original
df_tiendas = df_tiendas.merge(promedio_ordenes_por_mes, on='customer_id', how='left')

# Ver las primeras filas del DataFrame actualizado
print(df_tiendas.head())

   Unnamed: 0                                                 id  \
0          17  bytearray(b'\x00\x00w\x17\xae<@\xb4\x87-\x98\x...   
1          24  bytearray(b'\x00\x00\xb2\x8cP\x03D5\x85\xd0\x1...   
2          63  bytearray(b'\x00\x01\xaaN\xed\x19O<\x8d\xbfwT\...   
3         101  bytearray(b'\x00\x02\x97\xde\xd0\x99NP\x9dG\xd...   
4         153  bytearray(b'\x00\x04-\x18w\x0fE\xae\xa2\xbd3\x...   

   supplier_order_id  order_id     ship_at           shipped_at  supplier_id  \
0            2871968   2667640         NaN                  NaN         1403   
1             860931    793088  2022-01-31  2022-01-31 17:07:16            3   
2            1053490    967693  2022-03-09  2022-03-09 11:22:46            3   
3             608609    561574  2021-12-13  2021-12-13 20:26:10          408   
4            1768817   1630869  2022-06-29  2022-06-29 20:35:39          448   

   customer_id canceler supplier_reason  ...  \
0        26429      NaN             NaN  ...   
1        33446

# **% DESCUENTO TOTAL**

In [ ]:
# Agrupar por 'customer_id' y sumar los 'total' y 'discount_amount'
df_descuento_por_cliente = df_tiendas.groupby('customer_id').agg({
    'total': 'sum',
    'discount_amount': 'sum'
}).reset_index()

# Calcular el porcentaje total de descuento para cada 'customer_id'
df_descuento_por_cliente['porcentaje_descuento_total'] = (df_descuento_por_cliente['discount_amount'] / df_descuento_por_cliente['total']) * 100

# Unir este DataFrame con el DataFrame original usando 'customer_id'
df_tiendas = pd.merge(df_tiendas, df_descuento_por_cliente[['customer_id', 'porcentaje_descuento_total']], on='customer_id', how='left')

# **% DE ORDENES CON DESCUENTO**

In [ ]:
# Crear una columna booleana que indique si hubo descuento en la orden
df_tiendas['tuvo_descuento'] = df_tiendas['discount_amount'] > 0

# Agrupar por 'customer_id' y calcular la cantidad de órdenes con descuento y el total de órdenes
df_descuento_ordenes = df_tiendas.groupby('customer_id').agg({
    'tuvo_descuento': 'sum',  # Número de órdenes con descuento
    'order_id': 'count'       # Total de órdenes
}).reset_index()

# Calcular la proporción de órdenes con descuento
df_descuento_ordenes['proporcion_ordenes_descuentos'] = df_descuento_ordenes['tuvo_descuento'] / df_descuento_ordenes['order_id']

# Unir este DataFrame con el DataFrame original usando 'customer_id'
df_tiendas = pd.merge(df_tiendas, df_descuento_ordenes[['customer_id', 'proporcion_ordenes_descuentos']], on='customer_id', how='left')

In [ ]:
df_tiendas.columns

Index(['Unnamed: 0', 'id', 'supplier_order_id', 'order_id', 'ship_at',
       'shipped_at', 'supplier_id', 'customer_id', 'canceler',
       'supplier_reason', 'supplier_comment', 'customer_reason',
       'customer_comment', 'total', 'discount_amount', 'created_at',
       'confirmed_at', 'canceled_at', 'updated_at', 'credits_paid',
       'money_paid', 'service_fee', 'discount_id', 'delivery_lat',
       'delivery_lon', 'delivery_cost', 'final_amount', 'formatted_address',
       'customer_type_code', 'additional_info', 'promotions_amount',
       'state_id', 'postal_code', 'cantidad_proveedores',
       'porcentaje_max_supplier', 'year_month', 'promedio_ordenes_mes',
       'porcentaje_descuento_total', 'tuvo_descuento',
       'proporcion_descuentos'],
      dtype='object')

In [ ]:
# Seleccionar solo las columnas deseadas
df_final = df_tiendas[['customer_id', 'cantidad_ordenes','cantidad_proveedores', 'porcentaje_max_supplier',
                     'promedio_ordenes_mes', 'porcentaje_descuento_total', 'proporcion_ordenes_descuentos']]

In [ ]:
# Eliminar registros duplicados
df_final = df_final.drop_duplicates()

# Mostrar el DataFrame final sin duplicados
print(df_final.head())

   customer_id  cantidad_ordenes  cantidad_proveedores  \
0        26429               210                    26   
1        33446                42                    12   
2        22941               119                    23   
3        66442                39                     8   
4       135421                 6                     4   

   porcentaje_max_supplier  promedio_ordenes_mes  porcentaje_descuento_total  \
0                20.952381             16.153846                    5.922541   
1                19.047619              3.230769                    5.504166   
2                19.327731              7.933333                    7.543701   
3                46.153846              4.875000                    4.893332   
4                33.333333              2.000000                    3.730313   

   proporcion_ordenes_descuentos  
0                       0.800000  
1                       0.666667  
2                       0.966387  
3                       0.6666

In [ ]:
# Guardar el DataFrame final con solo las columnas seleccionadas y sin duplicados en un archivo CSV
df_final.to_csv('tiendas_caba_m2.csv', index=False)